# Kaggle project: Best Frameglass Order

## Introduction

We have 2 types of paintings: landscape and portrait. Each frameglass is able to hold 1 landscape painting or 2 portrait paintings. Each painting has a set of tags. The goal is to find the best order of frameglasses to maximize Global Robotic Satisfaction score.

Global Robotic Satisfaction score is a sum of Local Robotic Satisfaction. Local Robotic Satisfaction is calculated as a minimum of these 3 values:

- The number of common tags between two neighboring frameglasses

- The number of tags in the first painting but not in the second painting

- The number of tags in the second painting but not in the first painting

## Approach

We have 2 problems here:

1. How to divide the paintings into groups of 1 landscape painting and 2 portrait paintings

2. How to order the paintings in each group to maximize the Global Robotic Satisfaction score

These 2 problems can't be solved independently. We need to solve them together. We can use dynamic programming to solve this problem.

To maximize the Global Robotic Satisfaction score, we need to maximize the Local Robotic Satisfaction score between each pair of neighboring paintings. We can use dynamic programming to calculate the Local Robotic Satisfaction score between each pair of neighboring frameglasses.

Also, to maximize LRS score, we have to use frameglasses that have a half of the same tags, because the LRS score is $n/2$ at the best where $n$ is $min(tags1, tags2)$. It also means that it is better to assemble frameglasses with the most different tags.

In [34]:
'''Parse file data and return a list of dictionaries.
First row contains a number of columns, and the rest of the rows contain data:
- L or P in the first column indicates landscape or portrait orientation
- The second column contains a number of tags
- The rest of the columns contain the tags

@param file: path to the file
@return: a tuple containing the number of columns and a list of dictionaries'''
def parse_file(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    n = int(lines[0])
    data = []
    i = 0
    for line in lines[1:]:
        line = line.strip().split()
        orientation = line[0]
        tags = set(line[2:])
        data.append({'orientation': orientation, 'tags': tags, 'index': i})
        i += 1
    return n, data

In [35]:
n, data = parse_file('data/0_example.txt')
print(f'number of columns: {n}')
print(f'data: {data[:3]}')

number of columns: 4
data: [{'orientation': 'L', 'tags': {'animals', 'fear', 'war'}, 'index': 0}, {'orientation': 'P', 'tags': {'woman', 'smile'}, 'index': 1}, {'orientation': 'P', 'tags': {'pearl', 'woman'}, 'index': 2}]


In [36]:
# get only portrait orientation
portraits = [d for d in data if d['orientation'] == 'P']

# get only landscape orientation
landscapes = [d for d in data if d['orientation'] == 'L']

# sort by number of tags
portraits.sort(key=lambda x: len(x['tags']), reverse=True)
print(f'portrait: {portraits[:3]}')

portrait: [{'orientation': 'P', 'tags': {'woman', 'smile'}, 'index': 1}, {'orientation': 'P', 'tags': {'pearl', 'woman'}, 'index': 2}]


In [37]:
# make frameglasses for portrait paintings pairs
# try to pair paintings to maximize the number of tags in the frame
portrait_pairs = []
while len(portraits) > 1:
    pair = (portraits.pop(0), portraits.pop(0))
    tags_num = len(pair[0]['tags'].union(pair[1]['tags']))
    for i, p in enumerate(portraits):
        if len(p.get('tags')) < tags_num * 2:
            break
        if len(pair[0]['tags'].union(p['tags'])) > tags_num:
            p2 = p
            portraits.insert(0, pair[1])
            pair = (pair[0], p2)
            tags_num = len(pair[0]['tags'].union(p2['tags']))
    portrait_pairs.append(pair)
if portraits:
    p1 = portraits.pop()
    p2 = {'index': -1, 'orientation': 'P', 'tags': set()}
    portrait_pairs.append((p1, p2))
    
print(f'portrait pairs: {portrait_pairs[:3]}')
        

portrait pairs: [({'orientation': 'P', 'tags': {'woman', 'smile'}, 'index': 1}, {'orientation': 'P', 'tags': {'pearl', 'woman'}, 'index': 2})]


In [38]:
# make frameglasses for portrait paintings pairs
portrait_frames = [{'index': (pair[0]['index'], pair[1]['index']), 'orientation': 'P', 'tags': pair[0]['tags'].union(pair[1]['tags'])} for pair in portrait_pairs]

# assemble the final list of frames
frames = list()
frames.extend(portrait_frames)
frames.extend(landscapes)

# sort by number of tags
frames.sort(key=lambda x: len(x['tags']), reverse=True)
print(f'frames: {frames[:3]}')

frames: [{'index': (1, 2), 'orientation': 'P', 'tags': {'pearl', 'woman', 'smile'}}, {'orientation': 'L', 'tags': {'animals', 'fear', 'war'}, 'index': 0}, {'orientation': 'L', 'tags': {'raft', 'fear', 'survivors'}, 'index': 3}]


In [39]:
# hash for set
def hash_set(s):
    return hash(frozenset(s))

In [40]:
# make all sets of tags as numpy unique arrays
for frame in frames:
    frame['tags'] = frozenset([hash(tag) for tag in frame['tags']])
    n = 4
    chuncks = [frozenset(list(frame['tags'])[i:i+n]) for i in range(0, len(frame['tags']), n)]
    frame['tag_chuncks'] = chuncks

print(f'frames: {frames[:3]}')

frames: [{'index': (1, 2), 'orientation': 'P', 'tags': frozenset({-6511255215490526725, 5732252533558714048, 8065561845769783611}), 'tag_chuncks': [frozenset({-6511255215490526725, 5732252533558714048, 8065561845769783611})]}, {'orientation': 'L', 'tags': frozenset({609231729198667929, 2893920296272978417, 6979748917437063883}), 'index': 0, 'tag_chuncks': [frozenset({609231729198667929, 2893920296272978417, 6979748917437063883})]}, {'orientation': 'L', 'tags': frozenset({609231729198667929, 4808285081334926018, -7869181337856431928}), 'index': 3, 'tag_chuncks': [frozenset({609231729198667929, 4808285081334926018, -7869181337856431928})]}]


In [41]:
# LRS function
#lrs_memorized = dict()
def lrs(frame_1, frame_2):
    #if lrs_memorized.get((hash_set(frame_1['tags']), hash_set(frame_2['tags']))):
    #    return lrs_memorized[(hash_set(frame_1['tags']), hash_set(frame_2['tags']))]
    min_num = min(len(frame_1['tags']), len(frame_2['tags']))
    common_tags = frame_1['tags'] & (frame_2['tags'])
    # memoization
    #lrs_memorized[(hash_set(frame_1['tags']), hash_set(frame_2['tags']))] = result
    return min(len(common_tags), min_num - len(common_tags))

In [42]:
# order the frames to maximize LRS while traversing the list
frames_ordered = list()
frames_ordered.append(frames.pop(0))
while frames:
    frame_index = 0
    max_lrs = lrs(frames_ordered[-1], frames[frame_index])
    for i, f in enumerate(frames):
        if len(f['tags']) * 2 < max_lrs:
            break
        new_lrs = lrs(frames_ordered[-1], f)
        if max_lrs < new_lrs:
            max_lrs = new_lrs
            frame_index = i
    frames_ordered.append(frames.pop(frame_index))
    print(f'frames ordered: {len(frames_ordered)}', end='\r')

print(f'frames ordered: {frames_ordered[:3]}')

frames ordered: [{'index': (1, 2), 'orientation': 'P', 'tags': frozenset({-6511255215490526725, 5732252533558714048, 8065561845769783611}), 'tag_chuncks': [frozenset({-6511255215490526725, 5732252533558714048, 8065561845769783611})]}, {'orientation': 'L', 'tags': frozenset({609231729198667929, 2893920296272978417, 6979748917437063883}), 'index': 0, 'tag_chuncks': [frozenset({609231729198667929, 2893920296272978417, 6979748917437063883})]}, {'orientation': 'L', 'tags': frozenset({609231729198667929, 4808285081334926018, -7869181337856431928}), 'index': 3, 'tag_chuncks': [frozenset({609231729198667929, 4808285081334926018, -7869181337856431928})]}]


In [43]:
# calculate the sum of LRS
sum_lrs = 0
for i in range(len(frames_ordered) - 1):
    sum_lrs += lrs(frames_ordered[i], frames_ordered[i + 1])
print(f'sum of LRS: {sum_lrs}')

sum of LRS: 1


In [44]:
from time import time

# write the result to a file
with open(f'data/output/{time()}.txt', 'w') as f:
    f.write(f'{len(frames_ordered)}\n')
    for frame in frames_ordered:
        if frame['orientation'] == 'L':
            f.write(f'{frame["index"]}\n')
        else:
            i1 = frame['index'][0]
            i2 = frame['index'][1]
            if i2 == -1:
                f.write(f'{i1}\n')
            elif i1 == -1:
                f.write(f'{i2}\n')
            else:
                i1, i2 = sorted([i1, i2])
                f.write(f'{i1} {i2}\n')
print('done')

done
